In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math

from PIL import Image

In [ ]:
im = cv2.imread('sunflowers.jpg')
im_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
im_gray = im_gray.astype("float")
plt.imshow(im_gray,'gray')

# Exercise 8.1

In [ ]:
def gaussian1DKernel(sigma, length = 4):
    h = np.ceil(length*sigma)
    x = np.arange(-h,h+1)
    
    g = (1/(np.sqrt(2*np.pi*(sigma**2))))*np.exp(-x**2/(2*sigma**2))
    g /=g.sum()   
    #gx = -x*g/(sigma**2)

    g = g.reshape(-1, 1)
    #gx = gx.reshape(-1, 1)
    #return g,gx
    return g

In [ ]:
def scaleSpaced(im, sigma, n):
    im_scales = []
    for i in range(n):
        g = gaussian1DKernel(sigma*(2**i))
        im_gauss = cv2.filter2D(im,-1, g)
        im_gauss = cv2.filter2D(im_gauss,-1, g.T)
        im_scales.append(im_gauss)
    
    return im_scales

In [ ]:
im_scales = scaleSpaced(im_gray, 3,6)

In [ ]:
plt.figure(figsize=(30,10))
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(im_scales[i])

# Exercise 8.2

In [ ]:
def differenceOfGaussian(im, sigma, n):
    im_scales = scaleSpaced(im, sigma, n)
    DoG = []
    for i in range(n-1):
        DoGtemp = im_scales[i+1]-im_scales[i]
        DoG.append(DoGtemp)
        
    return DoG

In [ ]:
sigma = 3
n = 6
DoG = differenceOfGaussian(im_gray,sigma,n)
plt.figure(figsize=(30,10))
for i in range(n-1):
    plt.subplot(2,3,i+1)
    plt.imshow(DoG[i],'gray')

# Exercise 8.3

In [ ]:
def ownNMS(r,tau):
    row = len(r)
    column = len(r[0])
    DoGnMS = np.zeros((row,column))
    
    for i in range(1,row-1):
        for j in range(1,column-1):
            if r[i,j]> tau and r[i,j]>r[i+1,j] and r[i,j]>r[i,j+1] and r[i,j]>r[i-1,j] and r[i,j]>r[i,j-1] \
            and r[i,j]>r[i-1,j-1] and r[i,j]>r[i-1,j+1] and r[i,j]>r[i+1,j-1] and r[i,j]>r[i+1,j+1]:
                DoGnMS[i,j] = r[i,j]
            elif r[i,j]< -tau and r[i,j]<r[i+1,j] and r[i,j]<r[i,j+1] and r[i,j]<r[i-1,j] and r[i,j]<r[i,j-1] \
            and r[i,j]<r[i-1,j-1] and r[i,j]<r[i-1,j+1] and r[i,j]<r[i+1,j-1] and r[i,j]<r[i+1,j+1]:
                DoGnMS[i,j] = r[i,j]
    points = np.where(DoGnMS!=0) #find the local non-maximum suppression points in DoG[0]
    
    return points

In [ ]:
def scaleNMS(r,r0,points,tau):
    row = len(r)
    column = len(r[0])
    DoGnMS = np.zeros((row,column))
    
    for k in range(len(points[0])):
        i = points[0][k]
        j = points[1][k]
        if r[i,j]> tau and r[i,j]>r0[i,j] and r[i,j]>r0[i+1,j] and r[i,j]>r0[i,j+1] and r[i,j]>r0[i-1,j] and r[i,j]>r0[i,j-1] \
            and r[i,j]>r0[i-1,j-1] and r[i,j]>r0[i-1,j+1] and r[i,j]>r0[i+1,j-1] and r[i,j]>r0[i+1,j+1]:
                DoGnMS[i,j] = r[i,j]
        elif r[i,j]< -tau and r[i,j]<r0[i+1,j] and r[i,j]<r0[i,j+1] and r[i,j]<r0[i-1,j] and r[i,j]<r0[i,j-1] \
        and r[i,j]<r0[i-1,j-1] and r[i,j]<r0[i-1,j+1] and r[i,j]<r0[i+1,j-1] and r[i,j]<r0[i+1,j+1]:
            DoGnMS[i,j] = r[i,j]
    points = np.where(DoGnMS!=0) 
    
    return points

In [ ]:
def nonMaximumSuppression(DoG, threshold):
    pointsall = []
    valall = []
    for i in range(1,len(DoG)-1):
        r = DoG[i]
        r0 = DoG[i-1]
        r1 = DoG[i+1]
        points = ownNMS(r,threshold)
        points = scaleNMS(r,r0,points,threshold)
        points = scaleNMS(r,r1,points,threshold)
        val =abs(DoG[i][points[0][:],points[1][:]])
        
        pointsall.append(points)
        valall.append(val)
    return pointsall,valall

In [ ]:
pointsall,valall = nonMaximumSuppression(DoG,30)

In [ ]:
im = cv2.imread('sunflowers.jpg')
im1 = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)

In [ ]:
plt.figure(figsize=(20,20))
for i in range(len(pointsall)):
    points = pointsall[i]
    val = valall[i]
    for i in range(len(points[0])):
        cv2.circle(im1, (points[1][i], points[0][i]),int(val[i]*0.5) , color=(255, 0, 0), thickness=2)

plt.imshow(im1)

# Exercise 8.4

In [ ]:
#this is an lossy method....how to transform lossless?
def transformIm(im, theta, s):   
    (h, w) = im.shape[:2] 
    center = (w // 2, h // 2) 
    M = cv2.getRotationMatrix2D(center,theta , 1.0) 
    rotated = cv2.warpAffine(im, M, (w, h)) 
    scaled = cv2.resize(rotated, (int(w / s), int(h / s)))
    
    return scaled

In [ ]:
im_r3 = transformIm(im_gray, 30,1)
plt.imshow(im_r3,"gray")

# Exercise 8.5

In [ ]:
im = cv2.imread('sunflowers.jpg')
im_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)

In [ ]:
sift = cv2.SIFT_create(nfeatures=0, nOctaveLayers=3, contrastThreshold=0.1, edgeThreshold=5, sigma=1.6)
keypoints, descriptors = sift.detectAndCompute(im_gray, None)
img = cv2.drawKeypoints(image=im_gray, outImage=im, keypoints=keypoints,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.figure(figsize=(20,20))
plt.imshow(img)

In [ ]:
im1 = cv2.imread('sunflowers.jpg')
im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
im2 = transformIm(im1, 30,1)
hmerge = np.hstack((im1, im2))
plt.figure(figsize=(20,20))
plt.imshow(hmerge,"gray")

In [ ]:
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(im1, None)
kp2, des2 = sift.detectAndCompute(im2, None)
matcher = cv2.BFMatcher()
matches = matcher.knnMatch(des1, des2, k=2)  # k=2, find two closest
h1, w1 = im1.shape[:2]
h2, w2 = im2.shape[:2]

good_match = []
for m, n in matches:
    if m.distance < 0.4*n.distance:    #ratio should between 0.4~0.8 in oreder to get good result
        good_match.append([m])
        
out_img1 = cv2.drawMatchesKnn(im1, kp1, im2, kp2, good_match[:20], hmerge,flags = 2)  #only plot 20lines

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(out_img1)